### Import necessary libraries and packages

In [157]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
import webbrowser
from tqdm.notebook import tqdm_notebook, tqdm
from bs4 import BeautifulSoup
import time

### Get authentication code from redirect_uri

In [16]:

#stackapps 
client_id = #"stackapp_clien_id"

redirect_uri = "https://stackexchange.com/oauth/login_success"
scope = "no_expiry"
auth_url = f"https://stackoverflow.com/oauth?client_id={client_id}&scope={scope}&redirect_uri={redirect_uri}"

import subprocess

auth_url = f"https://stackoverflow.com/oauth?client_id={client_id}&scope={scope}&redirect_uri={redirect_uri}"
#subprocess.run(["open", auth_url])  # On macOS
subprocess.run(["xdg-open", auth_url])  # On Linux
#subprocess.run(["start", auth_url], shell=True)  # On Windows




/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://stackoverflow.com/oauth?client_id=29766&scope=no_expiry&redirect_uri=https://stackexchange.com/oauth/login_success'


CompletedProcess(args=['xdg-open', 'https://stackoverflow.com/oauth?client_id=29766&scope=no_expiry&redirect_uri=https://stackexchange.com/oauth/login_success'], returncode=3)

In [ ]:

client_secret = "stackapp client_secret"
redirect_uri = "https://stackexchange.com/oauth/login_success"

#authentication code from redirect_uri
code = "authentication code"

# Exchange authorization code for access token
token_url = "https://stackoverflow.com/oauth/access_token"
data = {
    'client_id': client_id,
    'client_secret': client_secret,
    'code': code,
    'redirect_uri': redirect_uri,
}

response = requests.post(token_url, data=data)

# Parse the access token from the response
access_token = response.text.split('=')
print("Access Token:", access_token)


### Class to scrap stackexchange data

In [124]:
class StackExchangeScrapper():
    def __init__(self,access_token: object, app_key: object):
        self.access_token = access_token
        self.app_key = app_key
        self.qna_df = ""
    

    def __get_total_question(self, tag, site):
        # Define the base URL for the StackExchange API
        url = "https://api.stackexchange.com/2.3/questions"


        params = {
            "tagged": tag,  # Tag for pandas-related questions
            "site": site,
            "filter": "total",  # Custom filter to include question and accepted answers
            'access_token': self.access_token,
            'key': self.app_key }
                            

        # Make the request
        response = requests.get( url, params=params)
        data = response.json()

        # Extract the total number of questions
        total_questions = data['total']

        # Output the result
        #print(f"Total number of questions tagged with 'python': {total_questions}")
        return total_questions
    
    def __get_question_answer(self, tag, site, page, pagesize):
        # Define the API URL for pandas questions
        url = "https://api.stackexchange.com/2.3/questions"

        # Parameters for the API call
        params = {
            "order": "desc",
            "sort": "votes",
            "tagged": tag,  # Tag for pandas-related questions
            "site": site,
            #"fromdate":"1451606400",
            "filter": "!-tsS42nh",  # Custom filter to exclude images
            "page":page,
            'pagesize': pagesize,            # Number of questions per request
            'access_token': self.access_token,
            'key': self.app_key
        }

        # Send a GET request to the API
        response = requests.get(url, params=params)
        questions_list = []

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()

            # Create a list to store questions and their details
            questions_answer_list = []

            # Loop through each question in the response
            for question in data['items']:
                if question['is_answered']:
                    if 'accepted_answer_id' in list(question.keys()):
                        question_data= {}
                        for field in ['title', 'link', 'tags', 'question_id', 'is_answered', 'accepted_answer_id',
                                      'view_count', 'answer_count', 'score', 'last_activity_date', 'creation_date'
                                       ]:

                            question_data[f"q_{field}"] = f"{question[field]}"
                        question_data.update(self.__get_accepted_answer(tag, site, question['accepted_answer_id']))
                        questions_answer_list.append(question_data)


            # Convert the list of questions into a Pandas DataFrame for easy manipulation
            #self.qna_df = pd.DataFrame(questions_answer_list)

            return questions_answer_list
        else:
            print(f"Failed to fetch data: {response.status_code}")
            
    # Function to get the accepted answer for a specific question
    def __get_accepted_answer(self, tag, site, answer_id):
        url = f"https://api.stackexchange.com/2.3/answers/{answer_id}"
        params = {
            #"order": "desc",
            #"sort": "votes",
            "tagged": tag,
            "site": site , # Cross Validated (stats.stackexchange.com)
            "filter": "!9_bDE(fI5", #"!-tsS42nh" # Custom filter to include answer body and is_accepted field
            'access_token': self.access_token,
            'key': self.app_key
        }

        # Make the request to the Stack Exchange API
        response = requests.get(url, params=params)
        #return response

        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            # Look for the accepted answer in the response
            answer_data = {}
            for answer in data['items']:
                for field in ['score',  'creation_date', 'body']:
                    if field == 'body':
                        answer_data[f"a_answer"] = self.__get_text_hyperlink(answer[field])
                    else:
                        answer_data[f"a_{field}"] = answer[field]


                return answer_data
            return None  # No accepted answer found
        else:
            print(f"Failed to fetch data: {response.status_code}")
            return None
    
    def __get_text_hyperlink(self, html_body):
    
        # Parse the HTML content
        soup = BeautifulSoup(html_body, 'html.parser')

        # Iterate over all anchor tags and replace them with "text (link)"
        for a in soup.find_all('a', href=True):
            a.replace_with(f"{a.get_text()} ({a['href']})")

        for img in soup.find_all('img'):
            img.decompose() 
         # Remove <a> tags that contain <img> tags
        for link in soup.find_all('a'):
            if link.find('img'):
                link.decompose()  # Removes the entire <a> tag if it contains an <img> tag


        # Get the resulting text including hyperlinks
        clean_text = soup.get_text()

        return clean_text
    
    def fetch_specified_results(self, tag, site, desired_count):
        total_count = self.__get_total_question(tag, site)
        if desired_count > total_count:
            print(f"Warning: Desired count ({desired_count}) exceeds available questions ({total_count}). Adjusting to {total_count}.")
            return None
        results = []
        page = 1
        pagesize = 10
        pbar = tqdm(total=desired_count,desc="Scrapping Data",initial=1)
        while len(results) < desired_count:
            # Fetch Q&A pairs from the current page
            qna_pairs = self.__get_question_answer(tag, site, page, pagesize)

            # Add the fetched results to the list
            results.extend(qna_pairs)

            
            pbar.update( len(qna_pairs))
            page += 1  # Move to the next page
            

            
        #pbar.update(desired_count - pbar.n)
        pbar.set_postfix({"status": "Completed"})
        pbar.close()
        
        # Return exactly the desired number of results
        self.qna_df = pd.DataFrame(results[:desired_count])
        #return self.qna_df 
        
    
    

    

In [53]:
#generated access_token from oauth
access_token = "access_token"

#stackapps
app_key = 'app_key'

### Create class instance

In [125]:
qna = StackExchangeScrapper(access_token,app_key)

### Scrape data

In [ ]:
qna.fetch_specified_results('neural_networks', 'stats',100)

### Pre-processing scrapped data using google gemini llm

In [110]:
pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.


In [129]:
import google.generativeai as genai
import os

genai.configure(api_key=secret_value_0)

### Build prompt to generate summary

In [153]:
def build_prompt(query):
    txt = """Please read the entire paragraph below and summarize it concisely and comprehensively. The summary must be no more than **350 words**. Use your skills and understanding to interpret the main ideas creatively while ensuring the response is brief and focused.

{info}

"""
                      
    
        
    txt = txt.format(info=query)
    #prompt = tokenizer.apply_chat_template(prompt_template, add_generation_prompt=True, return_dict=True, return_tensors="pt").to(model.device)

    return txt


### Define model

In [ ]:
model = genai.GenerativeModel("models/gemini-1.0-pro")

### Get summary for answers

In [ ]:
llm_answer_summary = []
for idx, answer in tqdm(enumerate(qna.qna_df['a_answer'])):
    response = model.generate_content(build_prompt(answer))
    llm_answer_summary.append(response.text)
    # Pause for 60 seconds after every 60 iterations
    if (idx + 1) % 15 == 0:
        print("Pausing for 60 seconds...")
        time.sleep(60)


In [161]:
qna.qna_df['llm_answer_summary'] = llm_answer_summary

### Actual answer

In [164]:
qna.qna_df.tail(1)['a_answer'][99]

'DropOut and DropConnect are both methods intended to prevent "co-adaptation" of units in a neural network. In other words, we want units to independently extract features from their inputs instead of relying on other neurons to do so. \nSuppose we have a multilayered feedforward network like this one (the topology doesn\'t really matter). We\'re worried about the yellow hidden units in the middle layer co-adapting. \n (https://i.sstatic.net/DfKTA.png)\n DropOut \nTo apply DropOut, we randomly select a subset of the units and clamp their output to zero, regardless of the input; this effectively removes those units from the model. A different subset of units is randomly selected every time we present a training example. \nBelow are two possible network configurations. On the first presentation (left), the 1st and 3rd units are disabled, but the 2nd and 3rd units have been randomly selected on a subsequent presentation. At test time, we use the complete network but rescale the weights to

### LLM summary

In [165]:
qna.qna_df.tail(1)['llm_answer_summary'][99]

'DropOut and DropConnect are regularization techniques in neural networks that aim to prevent co-adaptation of units. DropOut randomly sets a subset of units to zero during training, while DropConnect disables individual weights.\n\nBoth methods simulate training multiple networks simultaneously and averaging their outputs during testing. DropConnect allows for a greater variety of network configurations than DropOut.\n\nThe effectiveness of DropOut and DropConnect stems from their ability to promote the independent feature extraction of units, preventing over-reliance on other neurons. They achieve this by introducing randomness during training, forcing the network to learn robust representations.\n\nDropConnect is a generalization of DropOut, producing similar outcomes on an individual trial but offering more possible network configurations due to the increased number of connections compared to units. Both methods have proven effective in improving the robustness and generalization o

In [166]:
qna.qna_df.to_csv("Stackoverflow_data(neural_networks_stats)_pre_processed_Gemini_LLM.csv",index=False)